In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [25]:
NCPATH = "/work3/s174159/LLM_Thesis/data/No_Constraints/Combined/combined_dataset_corrected.json"
CPATH = "/work3/s174159/LLM_Thesis/data/Constraints/Combined/combined_dataset_corrected.json"
DPATH = "/work3/s174159/LLM_Thesis/data/No_Constraints/DocRED_Distant/combined_dataset_corrected.json"
JPATH = "/work3/s174159/LLM_Thesis/data/No_Constraints/DocRED_Joint/combined_dataset_corrected.json"
RPATH = "/work3/s174159/LLM_Thesis/data/No_Constraints/REBEL/combined_dataset_corrected.json"

In [16]:
NCDF = pd.read_json(NCPATH)
CDF = pd.read_json(CPATH)
JDF = pd.read_json(JPATH)
DDF = pd.read_json(DPATH)
DDF = DDF[DDF['original_file_path'].str.contains("distant", na=False)]



In [46]:
NCRAWDF = pd.read_json(NCPATH)

In [17]:
NCDF = pd.concat([NCDF, DDF, JDF], ignore_index=True)
CDF = pd.concat([CDF, DDF, JDF], ignore_index=True)

In [18]:
paths = []
for i, row in DDF.iterrows():
    file_path = row['original_file_path']
    paths.append(file_path)


In [19]:
pd.set_option('max_colwidth', 800)

In [20]:
NCDF

,org_dataset,title,domains,sents,vertexSet,labels,original_file_path
0,SciERC,X96-1059,science,"[[Recognition, of, proper, nouns, in, Japanese, text, has, been, studied, as, a, part, of, the, more, general, problem, of, morphological, analysis, in, Japanese, text, processing, -LRB-, -LSB-, 1, -RSB-, -LSB-, 2, -RSB-, -RRB-, .], [It, has, also, been, studied, in, the, framework, of, Japanese, information, extraction, -LRB-, -LSB-, 3, -RSB-, -RRB-, in, recent, years, .], [Our, approach, to, the, Multi-lingual, Evaluation, Task, -LRB-, MET, -RRB-, for, Japanese, text, is, to, consider, the, given, task, as, a, morphological, analysis, problem, in, Japanese, .], [Our, morphological, analyzer, has, done, all, the, necessary, work, for, the, recognition, and, classification, of, proper, names, ,, numerical, and, temporal, expressions, ,, i.e., Named, Entity, -LRB-, NE, -RRB-, items, in,...","[[{'name': 'Recognition of proper nouns', 'pos': [0, 3], 'sent_id': 0, 'type': 'Task'}], [{'name': 'morphological analysis', 'pos': [19, 20], 'sent_id': 0, 'type': 'Task'}], [{'name': 'proper nouns', 'pos': [2, 3], 'sent_id': 0, 'type': 'OtherScientificTerm'}], [{'name': 'Japanese text', 'pos': [5, 6], 'sent_id': 0, 'type': 'Material'}], [{'name': 'Japanese text processing', 'pos': [22, 24], 'sent_id': 0, 'type': 'Task'}], [{'name': 'Japanese information extraction', 'pos': [9, 11], 'sent_id': 1, 'type': 'Task'}], [{'name': 'It', 'pos': [0, 0], 'sent_id': 1, 'type': 'Other'}], [{'name': 'approach', 'pos': [1, 1], 'sent_id': 2, 'type': 'Other'}], [{'name': 'Multi-lingual Evaluation Task -LRB- MET -RRB-', 'pos': [4, 9], 'sent_id': 2, 'type': 'Task'}], [{'name': 'morphological analysis pr...","[{'r': 'Part of', 'h': 0, 't': 1, 'evidence': [0]}, {'r': 'Part of', 'h': 2, 't': 3, 'evidence': [0]}, {'r': 'Used for', 'h': 1, 't': 4, 'evidence': [0]}, {'r': 'Used for', 'h': 5, 't': 6, 'evidence': [1]}, {'r': 'Used for', 'h': 7, 't': 8, 'evidence': [2]}, {'r': 'Used for', 'h': 8, 't': 3, 'evidence': [2]}, {'r': 'Used for', 'h': 9, 't': 10, 'evidence': [2]}, {'r': 'Used for', 'h': 11, 't': 9, 'evidence': [2]}, {'r': 'Used for', 'h': 12, 't': 13, 'evidence': [3]}, {'r': 'Hyponym of', 'h': 14, 't': 15, 'evidence': [3]}, {'r': 'Part of', 'h': 14, 't': 3, 'evidence': [3]}, {'r': 'Used for', 'h': 16, 't': 17, 'evidence': [5]}, {'r': 'Part of', 'h': 18, 't': 16, 'evidence': [5]}, {'r': 'Conjunction', 'h': 18, 't': 19, 'evidence': [5]}, {'r': 'Part of', 'h': 19, 't': 16, 'evidence': [5]}, ...",/work3/s174159/LLM_Thesis/data/raw_data/SciERC/test.json
1,SciERC,ICCV_2001_47_abs,science,"[[We, propose, to, incorporate, a, priori, geometric, constraints, in, a, 3, --, D, stereo, reconstruction, scheme, to, cope, with, the, many, cases, where, image, information, alone, is, not, sufficient, to, accurately, recover, 3, --, D, shape, .], [Our, approach, is, based, on, the, iterative, deformation, of, a, 3, --, D, surface, mesh, to, minimize, an, objective, function, .], [We, show, that, combining, anisotropic, meshing, with, a, non-quadratic, approach, to, regularization, enables, us, to, obtain, satisfactory, reconstruction, results, using, triangulations, with, few, vertices, .], [Structural, or, numerical, constraints, can, then, be, added, locally, to, the, reconstruction, process, through, a, constrained, optimization, scheme, .], [They, improve, the, reconstruction, ...","[[{'name': 'priori geometric constraints', 'pos': [5, 7], 'sent_id': 0, 'type': 'OtherScientificTerm'}], [{'name': '3 -- D stereo reconstruction scheme', 'pos': [10, 15], 'sent_id': 0, 'type': 'Method'}], [{'name': 'image information', 'pos': [23, 24], 'sent_id': 0, 'type': 'OtherScientificTerm'}], [{'name': '3 -- D shape', 'pos': [32, 35], 'sent_id': 0, 'type': 'OtherScientificTerm'}], [{'name': 'iterative deformation of a 3 -- D surface mesh', 'pos': [6, 14], 'sent_id': 1, 'type': 'Method'}], [{'name': 'approach', 'pos': [1, 1], 'sent_id': 1, 'type': 'Other'}], [{'name': 'objective function', 'po

In [34]:
import numpy as np
from collections import Counter

def compute_data_stats(df):
    datasets_stats = {}

    # Get unique datasets
    unique_datasets = df['org_dataset'].unique()

    for dataset in unique_datasets:
        sDF = df[df['org_dataset'] == dataset]
        vertexSet = []
        relations = []
        total_entities = 0
        total_relations = 0

        # Metrics
        total_records = len(sDF)

        # Sentence metrics
        sentence_lengths = [len(sent) for sents in sDF['sents'] for sent in sents]
        avg_sent_length = np.mean(sentence_lengths) if sentence_lengths else 0
        max_sent_length = np.max(sentence_lengths) if sentence_lengths else 0
        min_sent_length = np.min(sentence_lengths) if sentence_lengths else 0

        # Number of sentences per record
        num_sents_per_record = [len(sents) for sents in sDF['sents']]
        avg_sents_per_record = np.mean(num_sents_per_record) if num_sents_per_record else 0
        max_sents_per_record = np.max(num_sents_per_record) if num_sents_per_record else 0
        min_sents_per_record = np.min(num_sents_per_record) if num_sents_per_record else 0

        # Check if it's a document-based dataset
        is_document_based = "Yes" if max_sents_per_record > 1 else "No"

        # Collect entity types and relation types
        for _, row in sDF.iterrows():
            # Entity types in vertexSet
            for ver in row['vertexSet']:
                total_entities += len(ver)
                for v in ver:
                    vertexSet.append(v['type'])
            # Relation types in labels
            total_relations += len(row['labels'])
            for rel in row['labels']:
                relations.append(rel['r'])

        # Entity types metrics
        entity_type_counts = Counter(vertexSet)
        avg_entity_types_per_record = len(vertexSet) / total_records if total_records > 0 else 0
        max_entity_types_per_record = max([len(vs) for vs in sDF['vertexSet']])
        min_entity_types_per_record = min([len(vs) for vs in sDF['vertexSet']])
        unique_entity_types = len(entity_type_counts)

        # Relation types metrics
        relation_type_counts = Counter(relations)
        avg_relation_types_per_record = len(relations) / total_records if total_records > 0 else 0
        max_relation_types_per_record = max([len(l) for l in sDF['labels']])
        min_relation_types_per_record = min([len(l) for l in sDF['labels']])
        unique_relation_types = len(relation_type_counts)

        # Store the metrics for the current dataset
        datasets_stats[dataset] = {
            'Total Records': total_records,
            'Avg Sentence Length (tokens)': avg_sent_length,
            'Max Sentence Length (tokens)': max_sent_length,
            'Min Sentence Length (tokens)': min_sent_length,
            'Avg Sentences per Record': avg_sents_per_record,
            'Max Sentences per Record': max_sents_per_record,
            'Min Sentences per Record': min_sents_per_record,
            'Is Document-Based': is_document_based,
            'Total Entities': total_entities,
            'Avg Entity Types per Record': avg_entity_types_per_record,
            'Max Entity Types per Record': max_entity_types_per_record,
            'Min Entity Types per Record': min_entity_types_per_record,
            'Unique Entity Types': unique_entity_types,
            'Entity Type Counts': dict(entity_type_counts),
            'Total Relations': total_relations,
            'Avg Relation Types per Record': avg_relation_types_per_record,
            'Max Relation Types per Record': max_relation_types_per_record,
            'Min Relation Types per Record': min_relation_types_per_record,
            'Unique Relation Types': unique_relation_types,
            'Relation Type Counts': dict(relation_type_counts)
        }
    
    return datasets_stats


In [33]:
compute_data_stats(NCDF)

{'SciERC': {'Total Records': 500,
  'Avg Sentence Length (tokens)': 24.31484927428359,
  'Max Sentence Length (tokens)': 101,
  'Min Sentence Length (tokens)': 1,
  'Avg Sentences per Record': 5.374,
  'Max Sentences per Record': 12,
  'Min Sentences per Record': 1,
  'Is Document-Based': 'Yes',
  'Total Entities': 7236,
  'Avg Entity Types per Record': 14.472,
  'Max Entity Types per Record': 36,
  'Min Entity Types per Record': 0,
  'Unique Entity Types': 6,
  'Entity Type Counts': {'Task': 1203,
   'OtherScientificTerm': 2039,
   'Material': 717,
   'Other': 1065,
   'Method': 1901,
   'Metric': 311},
  'Total Relations': 4648,
  'Avg Relation Types per Record': 9.296,
  'Max Relation Types per Record': 31,
  'Min Relation Types per Record': 0,
  'Unique Relation Types': 7,
  'Relation Type Counts': {'Part of': 269,
   'Used for': 2437,
   'Hyponym of': 409,
   'Conjunction': 582,
   'Feature of ': 264,
   'Evaluate for': 454,
   'Compare': 233}},
 'CrossRE': {'Total Records': 4850,

In [35]:
def dict_to_latex_table_by_metric(datasets_stats):
    latex_code = []
    
    # Get list of datasets
    dataset_names = list(datasets_stats.keys())
    
    # Start the table
    latex_code.append(r"\begin{table}[ht]")
    latex_code.append(r"\centering")
    latex_code.append(r"\begin{tabular}{|l" + "|c" * len(dataset_names) + "|}")
    latex_code.append(r"\hline")
    
    # Header row (Dataset names)
    header_row = "Metric"
    for dataset in dataset_names:
        header_row += f" & {dataset}"
    latex_code.append(header_row + r" \\ \hline")
    
    # List of metrics
    metrics = [
        'Total Records', 
        'Avg Sentence Length (tokens)', 
        'Max Sentence Length (tokens)', 
        'Min Sentence Length (tokens)', 
        'Avg Sentences per Record', 
        'Max Sentences per Record', 
        'Min Sentences per Record', 
        'Is Document-Based',
        'Total Entities',
        'Avg Entity Types per Record',
        'Max Entity Types per Record',
        'Min Entity Types per Record',
        'Unique Entity Types',
        'Total Relations',
        'Avg Relation Types per Record', 
        'Max Relation Types per Record', 
        'Min Relation Types per Record', 
        'Unique Relation Types'
    ]
    
    # Add a row for each metric
    for metric in metrics:
        row = metric
        for dataset in dataset_names:
            value = datasets_stats[dataset].get(metric, 'N/A')
            if isinstance(value, float):
                row += f" & {value:.2f}"
            else:
                row += f" & {value}"
        latex_code.append(row + r" \\ \hline")
    
    # End the table
    latex_code.append(r"\end{tabular}")
    latex_code.append(r"\caption{Dataset Statistics by Metric}")
    latex_code.append(r"\label{tab:dataset_stats_by_metric}")
    latex_code.append(r"\end{table}")
    
    return "\n".join(latex_code)

# Example usage:
# latex_code = dict_to_latex_table_by_metric(stats)
# print(latex_code)


In [36]:
code = dict_to_latex_table_by_metric(compute_data_stats(NCDF))
print(code)

\begin{table}[ht]
\centering
\begin{tabular}{|l|c|c|c|c|c|c|c|}
\hline
Metric & SciERC & CrossRE & NYT & CoNLL04 & ReDocRED & DocRED_Distant & DocRED_Joint \\ \hline
Total Records & 500 & 4850 & 66196 & 1441 & 4053 & 101873 & 4008 \\ \hline
Avg Sentence Length (tokens) & 24.31 & 36.36 & 37.70 & 29.05 & 24.87 & 24.55 & 24.87 \\ \hline
Max Sentence Length (tokens) & 101 & 124 & 100 & 118 & 180 & 197 & 180 \\ \hline
Min Sentence Length (tokens) & 1 & 3 & 4 & 2 & 6 & 1 & 6 \\ \hline
Avg Sentences per Record & 5.37 & 1.00 & 1.00 & 1.00 & 7.98 & 8.13 & 7.97 \\ \hline
Max Sentences per Record & 12 & 1 & 1 & 1 & 25 & 36 & 25 \\ \hline
Min Sentences per Record & 1 & 1 & 1 & 1 & 3 & 3 & 3 \\ \hline
Is Document-Based & Yes & No & No & No & Yes & Yes & Yes \\ \hline
Total Entities & 7236 & 26910 & 203674 & 5349 & 105688 & 2558350 & 104367 \\ \hline
Avg Entity Types per Record & 14.47 & 5.55 & 3.08 & 3.71 & 26.08 & 25.11 & 26.04 \\ \hline
Max Entity Types per Record & 36 & 32 & 20 & 24 & 42 & 42 & 

In [39]:
import numpy as np
from collections import Counter

def compute_global_data_stats(df):
    # Initialize variables for global stats
    vertexSet = []
    relations = []
    total_entities = 0
    total_relations = 0

    # Metrics
    total_records = len(df)

    # Sentence metrics
    sentence_lengths = [len(sent) for sents in df['sents'] for sent in sents]
    avg_sent_length = np.mean(sentence_lengths) if sentence_lengths else 0
    max_sent_length = np.max(sentence_lengths) if sentence_lengths else 0
    min_sent_length = np.min(sentence_lengths) if sentence_lengths else 0

    # Number of sentences per record
    num_sents_per_record = [len(sents) for sents in df['sents']]
    avg_sents_per_record = np.mean(num_sents_per_record) if num_sents_per_record else 0
    max_sents_per_record = np.max(num_sents_per_record) if num_sents_per_record else 0
    min_sents_per_record = np.min(num_sents_per_record) if num_sents_per_record else 0

    # Check if it's a document-based dataset
    is_document_based = "Yes" if max_sents_per_record > 1 else "No"

    # Collect entity types and relation types
    for _, row in df.iterrows():
        # Entity types in vertexSet
        for ver in row['vertexSet']:
            total_entities += len(ver)
            for v in ver:
                vertexSet.append(v['type'])
        # Relation types in labels
        total_relations += len(row['labels'])
        for rel in row['labels']:
            relations.append(rel['r'])

    # Entity types metrics
    entity_type_counts = Counter(vertexSet)
    avg_entity_types_per_record = len(vertexSet) / total_records if total_records > 0 else 0
    max_entity_types_per_record = max([len(vs) for vs in df['vertexSet']])
    min_entity_types_per_record = min([len(vs) for vs in df['vertexSet']])
    unique_entity_types = len(entity_type_counts)

    # Relation types metrics
    relation_type_counts = Counter(relations)
    avg_relation_types_per_record = len(relations) / total_records if total_records > 0 else 0
    max_relation_types_per_record = max([len(l) for l in df['labels']])
    min_relation_types_per_record = min([len(l) for l in df['labels']])
    unique_relation_types = len(relation_type_counts)

    # Store the global metrics
    global_stats = {
        'Total Records': total_records,
        'Avg Sentence Length (tokens)': avg_sent_length,
        'Max Sentence Length (tokens)': max_sent_length,
        'Min Sentence Length (tokens)': min_sent_length,
        'Avg Sentences per Record': avg_sents_per_record,
        'Max Sentences per Record': max_sents_per_record,
        'Min Sentences per Record': min_sents_per_record,
        'Is Document-Based': is_document_based,
        'Total Entities': total_entities,
        'Avg Entity Types per Record': avg_entity_types_per_record,
        'Max Entity Types per Record': max_entity_types_per_record,
        'Min Entity Types per Record': min_entity_types_per_record,
        'Unique Entity Types': unique_entity_types,
        'Entity Type Counts': dict(entity_type_counts),
        'Total Relations': total_relations,
        'Avg Relation Types per Record': avg_relation_types_per_record,
        'Max Relation Types per Record': max_relation_types_per_record,
        'Min Relation Types per Record': min_relation_types_per_record,
        'Unique Relation Types': unique_relation_types,
        'Relation Type Counts': dict(relation_type_counts)
    }
    
    return global_stats


In [41]:
def dict_to_latex_table_for_global_stats(global_stats):
    latex_code = []

    # Start the table
    latex_code.append(r"\begin{table}[ht]")
    latex_code.append(r"\centering")
    latex_code.append(r"\begin{tabular}{|l|c|}")
    latex_code.append(r"\hline")

    # Header row (just two columns: Metric and Value)
    latex_code.append("Metric & Value \\ \hline")

    # List of metrics in the same order as they are stored
    metrics = [
        'Total Records', 
        'Avg Sentence Length (tokens)', 
        'Max Sentence Length (tokens)', 
        'Min Sentence Length (tokens)', 
        'Avg Sentences per Record', 
        'Max Sentences per Record', 
        'Min Sentences per Record', 
        'Is Document-Based',
        'Total Entities',
        'Avg Entity Types per Record',
        'Max Entity Types per Record',
        'Min Entity Types per Record',
        'Unique Entity Types',
        'Total Relations',
        'Avg Relation Types per Record', 
        'Max Relation Types per Record', 
        'Min Relation Types per Record', 
        'Unique Relation Types'
    ]

    # Add a row for each metric
    for metric in metrics:
        value = global_stats.get(metric, 'N/A')
        if isinstance(value, float):
            latex_code.append(f"{metric} & {value:.2f} \\ \hline")
        else:
            latex_code.append(f"{metric} & {value} \\ \hline")

    # End the table
    latex_code.append(r"\end{tabular}")
    latex_code.append(r"\caption{Global Dataset Statistics}")
    latex_code.append(r"\label{tab:global_dataset_stats}")
    latex_code.append(r"\end{table}")
    
    return "\n".join(latex_code)

# Example usage:
# global_stats = compute_global_data_stats(NCDF)
# latex_code = dict_to_latex_table_for_global_stats(global_stats)
# print(latex_code)


In [47]:
code_global = dict_to_latex_table_for_global_stats(compute_global_data_stats(NCRAWDF))

In [48]:
print(code_global)

\begin{table}[ht]
\centering
\begin{tabular}{|l|c|}
\hline
Metric & Value \ \hline
Total Records & 77040 \ \hline
Avg Sentence Length (tokens) & 33.33 \ \hline
Max Sentence Length (tokens) & 180 \ \hline
Min Sentence Length (tokens) & 1 \ \hline
Avg Sentences per Record & 1.40 \ \hline
Max Sentences per Record & 25 \ \hline
Min Sentences per Record & 1 \ \hline
Is Document-Based & Yes \ \hline
Total Entities & 348857 \ \hline
Avg Entity Types per Record & 4.53 \ \hline
Max Entity Types per Record & 42 \ \hline
Min Entity Types per Record & 0 \ \hline
Unique Entity Types & 45 \ \hline
Total Relations & 258669 \ \hline
Avg Relation Types per Record & 3.36 \ \hline
Max Relation Types per Record & 142 \ \hline
Min Relation Types per Record & 0 \ \hline
Unique Relation Types & 131 \ \hline
\end{tabular}
\caption{Global Dataset Statistics}
\label{tab:global_dataset_stats}
\end{table}


In [38]:
rDF = pd.read_json(RPATH)

In [27]:
code = dict_to_latex_table_by_metric(compute_data_stats(rDF))
print(code)

\begin{table}[ht]
\centering
\begin{tabular}{|l|c|}
\hline
Metric & REBEL \\ \hline
Total Records & 2078186 \\ \hline
Avg Sentence Length (tokens) & 19.46 \\ \hline
Max Sentence Length (tokens) & 477 \\ \hline
Min Sentence Length (tokens) & 1 \\ \hline
Avg Sentences per Record & 4.29 \\ \hline
Max Sentences per Record & 76 \\ \hline
Min Sentences per Record & 1 \\ \hline
Is Document-Based & Yes \\ \hline
Total Entities & 20027969 \\ \hline
Avg Entity Types per Record & 9.64 \\ \hline
Max Entity Types per Record & 94 \\ \hline
Min Entity Types per Record & 2 \\ \hline
Unique Entity Types & 144305 \\ \hline
Total Relations & 5546924 \\ \hline
Avg Relation Types per Record & 2.67 \\ \hline
Max Relation Types per Record & 371 \\ \hline
Min Relation Types per Record & 1 \\ \hline
Unique Relation Types & 1081 \\ \hline
\end{tabular}
\caption{Dataset Statistics by Metric}
\label{tab:dataset_stats_by_metric}
\end{table}
